In [ ]:
!nvidia-smi

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import h5py
import onnxruntime as ort
import openvino as ov
import re

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from pprint import pprint

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, macro_f1_similarity, N_CLASSES_2021_2022, N_CLASSES_2021, comp_metric, N_CLASSES_XC_LIGIT_SHORTEN, N_CLASSES_XC_LIGIT_EVEN_SHORTEN
# from code_base.utils.constants import SAMPLE_RATE
from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.models import WaveCNNClasifier, WaveCNNAttenClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.utils.swa import avarage_weights, delete_prefix_from_chkp
from code_base.inefernce import BirdsInference
from code_base.utils import load_json, compose_submission_dataframe, groupby_np_array, stack_and_max_by_samples, write_json
from code_base.utils.metrics import score_numpy
%matplotlib inline


# Export Models

In [ ]:
!ls -lt ../logdirs/ | head -20

In [ ]:
# bird2id_source = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_PrevComp.json")
# bird2id_target = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json")

# id2bird_source = {v:k for k,v in bird2id_source.items()}
# id2bird_target = {v:k for k,v in bird2id_target.items()}

# REARRANGE_INDICES = np.array([
#     bird2id_source[id2bird_target[i]] for i in range(len(id2bird_target))
# ]).astype(int)

# STRICT_LOAD = False

# def prune_checkpoint_rule(inp_chkp):
#     inp_chkp["head.attention.weight"] = inp_chkp["head.attention.weight"][REARRANGE_INDICES]
#     inp_chkp["head.attention.bias"] = inp_chkp["head.attention.bias"][REARRANGE_INDICES]
    
#     inp_chkp["head.fix_scale.weight"] = inp_chkp["head.fix_scale.weight"][REARRANGE_INDICES]
#     inp_chkp["head.fix_scale.bias"] = inp_chkp["head.fix_scale.bias"][REARRANGE_INDICES]

#     return inp_chkp

In [ ]:
# len(bird2id_source)

In [ ]:
MODEL_CLASS = WaveCNNAttenClasifier
TRAIN_PERIOD = 5

In [ ]:
EXP_NAME = "tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_AdamW1e4_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromPrebs1_PseudoF2PT05MT01P04I2_OOFPI2M2"

conf_path = glob(f"../logdirs/{EXP_NAME}/code/*train_configs*.py")
assert len(conf_path) == 1
conf_path = conf_path[0]
!cat {conf_path}

In [ ]:
MODELS = [
    {
        "model_config": dict(
            backbone="tf_efficientnetv2_s_in21k",
            mel_spec_paramms={
                "sample_rate": 32000,
                "n_mels": 128,
                "f_min": 20,
                "n_fft": 2048,
                "hop_length": 512,
                "normalized": True,
            },
            head_config={
                "p": 0.5,
                "num_class": 206,
                "train_period": TRAIN_PERIOD,
                "infer_period": TRAIN_PERIOD,
                "output_type": "clipwise_pred_long",
            },
            exportable=True,
            fixed_amplitude_to_db=True
        ),
        "exp_name": "tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_AdamW1e4_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2ExtSWA_PseudoF2PT05MT01P04I2MOOFRev_AddRareBirdsNoLeak",
        "fold": [0, 1, 2, 3, 4],
        # "fold": [1, 3, 4],
        "chkp_name":"last.ckpt",
        "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
        "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
        "delete_prefix": "model.",
        "n_swa_models": 1,
        "model_output_key": None,
        # "prune_checkpoint_func": prune_checkpoint_rule
    },
    {
        "model_config": dict(
            backbone="tf_efficientnetv2_s_in21k",
            mel_spec_paramms={
                "sample_rate": 32000,
                "n_mels": 128,
                "f_min": 20,
                "n_fft": 2048,
                "hop_length": 512,
                "normalized": True,
            },
            head_config={
                "p": 0.5,
                "num_class": 206,
                "train_period": TRAIN_PERIOD,
                "infer_period": TRAIN_PERIOD,
                "output_type": "clipwise_pred_long",
            },
            exportable=True,
            fixed_amplitude_to_db=True
        ),
        "exp_name": "tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_AdamW1e4_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromPrebs1_PseudoF2PT05MT01P04I2_OOFPI2M2",
        "fold": [0, 1, 2, 3, 4],
        "chkp_name":"last.ckpt",
        "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
        "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
        "delete_prefix": "model.",
        "n_swa_models": 1,
        "model_output_key": None,
        # "prune_checkpoint_func": prune_checkpoint_rule
    },
    
    # {
    #     "model_config": dict(
    #         backbone="eca_nfnet_l0",
    #         mel_spec_paramms={
    #             "sample_rate": 32000,
    #             "n_mels": 128,
    #             "f_min": 20,
    #             "n_fft": 2048,
    #             "hop_length": 512,
    #             "normalized": True,
    #         },
    #         head_config={
    #             "p": 0.5,
    #             "num_class": 206,
    #             "train_period": TRAIN_PERIOD,
    #             "infer_period": TRAIN_PERIOD,
    #             "output_type": "clipwise_pred_long",
    #         },
    #         exportable=True,
    #         fixed_amplitude_to_db=True
    #     ),
    #     "exp_name": "eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromXCV2Best_PseudoF2PT05MT01P04I3_MinorOverSampleV1",
    #     "fold": [0, 1, 2, 3, 4],
    #     "chkp_name":"last.ckpt",
    #     "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
    #     "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
    #     "delete_prefix": "model.",
    #     "n_swa_models": 1,
    #     "model_output_key": None,
    #     # "prune_checkpoint_func": prune_checkpoint_rule
    # },
]

In [ ]:
def create_model_and_upload_chkp(
    model_class,
    model_config,
    model_device,
    model_chkp_root,
    model_chkp_basename=None,
    model_chkp_regex=None,
    delete_prefix=None,
    swa_sort_rule=None,
    n_swa_to_take=3,
    prune_checkpoint_func=None
):
    if model_chkp_basename is None:
        basenames = os.listdir(model_chkp_root)
        checkpoints = []
        for el in basenames:
            matches = re.findall(model_chkp_regex, el)
            if not matches:
                continue
            parsed_dict = {key: value for key, value in matches}
            parsed_dict["name"] = el
            checkpoints.append(parsed_dict)
        print("SWA checkpoints")
        pprint(checkpoints)
        checkpoints = sorted(checkpoints, key=swa_sort_rule)
        checkpoints = checkpoints[:n_swa_to_take]
        print("SWA sorted checkpoints")
        pprint(checkpoints)
        if len(checkpoints) > 1:
            checkpoints = [
                torch.load(os.path.join(model_chkp_root, el["name"]), map_location="cpu")["state_dict"] for el in checkpoints
            ]
            t_chkp = avarage_weights(
                nn_weights=checkpoints,
                delete_prefix=delete_prefix
            )
        else:
            chkp_path = os.path.join(model_chkp_root, checkpoints[0]["name"])
            print("vanilla model")
            print("Loading", chkp_path)
            t_chkp = torch.load(
                chkp_path, 
                map_location="cpu"
            )["state_dict"]
            if delete_prefix is not None:
                t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)
    else:
        chkp_path = os.path.join(model_chkp_root, model_chkp_basename)
        print("vanilla model")
        print("Loading", chkp_path)
        t_chkp = torch.load(
            chkp_path, 
            map_location="cpu"
        )["state_dict"]
        if delete_prefix is not None:
            t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)

    if prune_checkpoint_func is not None:
        t_chkp = prune_checkpoint_func(t_chkp)
    t_model = model_class(**model_config, device=model_device) 
    print("Missing keys: ", set(t_model.state_dict().keys()) - set(t_chkp))
    print("Extra keys: ",  set(t_chkp) - set(t_model.state_dict().keys()))
    t_model.load_state_dict(t_chkp, strict=False)
    t_model.eval()
    return t_model

In [ ]:
model = []
for config in MODELS:
    model.extend([create_model_and_upload_chkp(
        model_class=MODEL_CLASS,
        model_config=config['model_config'],
        model_device="cpu",
        model_chkp_root=f"../logdirs/{config['exp_name']}/fold_{m_i}/checkpoints",
        model_chkp_basename=config["chkp_name"] if config["swa_checkpoint_regex"] is None else None,
        model_chkp_regex=config.get("swa_checkpoint_regex"),
        swa_sort_rule=config.get("swa_sort_rule"),
        n_swa_to_take=config.get("n_swa_models", 3),
        delete_prefix=config.get("delete_prefix"),
        prune_checkpoint_func=config.get("prune_checkpoint_func")
    ) for m_i in config["fold"]])

In [ ]:
len(model)

In [ ]:
exportable_ensem = ONNXEnsemble(
    model_class=MODEL_CLASS,
    configs=list(chain(*[
        [deepcopy(config['model_config']) for _ in config["fold"]] for config in MODELS
    ])),
    extract_spec_ones=True
    # avarage_type="identity"
    # weights=[2.0] * 5 + [1.0] * 5
)
assert len(exportable_ensem.models) == len(model)
for model_id in range(len(model)):
    exportable_ensem.models[model_id].load_state_dict(model[model_id].state_dict())
    exportable_ensem.models[model_id].eval()
exportable_ensem.eval()
convert_to_onnx(
    model_to_convert=exportable_ensem,
    sample_input=torch.randn(5, TRAIN_PERIOD * 32_000),
    base_path=f"../logdirs/ebs_19__ebs_13/onnx_ensem_5first_folds",
    use_fp16=True,
    use_openvino=True,
    opset_version=12
)

In [ ]:
# !rm ../logdirs/convnext_small_fb_in22k_ft_in1k_384__convnextv2_tiny_fcmae_ft_in22k_in1k_384__eca_nfnet_l0_noval_v27_075Clipwise025TimeMax_GausMean -rf

In [ ]:
# !rm ../logdirs/convnext_small_fb_in22k_ft_in1k_384__convnextv2_tiny_fcmae_ft_in22k_in1k_384__eca_nfnet_l0_noval_v27_075Clipwise025TimeMax_GausMean -rf

In [ ]:
!ls -lt ../logdirs/ | head

# Check

In [ ]:
os.listdir(f"../logdirs/{EXP_NAME}/")

In [ ]:
core = ov.Core()
model = core.read_model(model=f"../logdirs/{EXP_NAME}/onnx_ensem_openvino_fp16/model_simpl.xml")
compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
sample = np.random.randn(5, 32000*5).astype(np.float16)

In [ ]:
compiled_model.output(0)

In [ ]:
%%time
output = compiled_model([sample])[compiled_model.output(0)]

In [ ]:
output.shape

In [ ]:
output.dtype

In [ ]:
sample = sample.astype(np.float16)

In [ ]:
onnx_model = ort.InferenceSession(
    f"../logdirs/{EXP_NAME}/onnx_ensem_fp16/model_simpl.onnx"
)

In [ ]:
wave = torch.from_numpy(np.random.randn(32000*5)).half()

In [ ]:
wave.numpy()

In [ ]:
%%time
out = onnx_model.run(
    None,
    {"input": sample}
)

# Emulate test

In [ ]:
ds_config_test = {
   # "root": "../data/train_audio",
    "root": "",
   "label_str2int_mapping_path": "../data/bird2int_2025.json",
   "n_cores": 64,
   "use_audio_cache": True,
   "test_mode": True,
   "segment_len": 5,
    "do_not_exceed_duration": True,
   "lookback": None,
   "lookahead": None,
    "step": 2.5,
    "sample_id": None,
    "late_normalize": True,
    "load_normalize": False,
    # "step": None,
    "validate_sr": 32_000
}
loader_config = {
    "batch_size": 24,
    "drop_last": False,
    "shuffle": False,
    "num_workers": 0,
}

In [ ]:
test_au_pathes = glob("../data/train_audio_soundscapes/*.ogg")
test_au_pathes = test_au_pathes[:10]

test_df = pd.DataFrame({
    "filename": test_au_pathes,
    "duration_s": [librosa.get_duration(filename=el) for el in test_au_pathes]
})
test_df

In [ ]:
# test_df = pd.read_csv(
#     "../data/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet11052025_hdf5_csa_fixedaudiometa_h5pyDur.csv"
# )

In [ ]:
# test_df = test_df.iloc[10:13]

In [ ]:
ds_test = WaveAllFileDataset(df=test_df, **ds_config_test)
loader_test = torch.utils.data.DataLoader(
    ds_test,
    **loader_config,
)

In [ ]:
len(ds_test) / 10

In [ ]:
60 / 5

In [ ]:
ds_test[0]

In [ ]:
1760000 / 32000

In [ ]:
1840000 / 32000

In [ ]:
ds_test.sampleidx_2_dfidx

In [ ]:
[ds_test[i][-1] for i in range(len(ds_test))]

In [ ]:
def compile_model(model_path_):
    core = ov.Core()
    compiled_model_ = core.read_model(model=model_path_)
    return core.compile_model(model=compiled_model_, device_name="CPU")

inference_class_onnx = BirdsInference(
    device="cpu",
    verbose_tqdm=True,
    use_sigmoid=False,
    use_compiled_fp16=False,
)
onnx_model = compile_model(
    "../logdirs/best_ensem_16052025/onnx_ensem_5first_folds_openvino_fp16/model_simpl.xml"
)
test_preds, test_dfidx, test_end = inference_class_onnx.predict_test_loader(
    nn_models=onnx_model,
    data_loader=loader_test,
    is_openvino_model=True,
)

In [ ]:
def aggregate_predictions(test_preds_, test_dfidx_, test_end_):
    unique_files = np.unique(test_dfidx_)
    agg_preds = []
    agg_dfidx = []
    agg_end = []

    for file_id in unique_files:
        mask = test_dfidx_ == file_id
        preds = test_preds_[mask]
        ends = test_end_[mask]

        if len(preds) < 1:
            continue

        num_preds = len(preds)
        file_agg_preds = []
        file_agg_ends = []

        for i in range(num_preds):
            components = []
            weights = []

            # t-1
            if i > 0:
                components.append(preds[i - 1])
                weights.append(0.25)
            # zero-padding is removed; only normalize used weights
            # t
            components.append(preds[i])
            weights.append(0.5)
            # t+1
            if i < num_preds - 1:
                components.append(preds[i + 1])
                weights.append(0.25)

            weights = np.array(weights)
            components = np.stack(components, axis=0)
            normalized_weights = weights / weights.sum()
            weighted_pred = (normalized_weights[:, None] * components).sum(axis=0)

            if i % 2 == 0:
                file_agg_preds.append(weighted_pred)
                file_agg_ends.append(ends[i])

        agg_preds.append(np.stack(file_agg_preds, axis=0))
        agg_dfidx.append(np.full(len(file_agg_preds), file_id))
        agg_end.append(np.array(file_agg_ends))

    return (
        np.concatenate(agg_preds, axis=0),
        np.concatenate(agg_dfidx, axis=0),
        np.concatenate(agg_end, axis=0),
    )

In [ ]:
test_preds_aggr, test_dfidx_aggr, test_end_aggr = aggregate_predictions(
    test_preds_=test_preds,
    test_dfidx_=test_dfidx,
    test_end_=test_end
)

In [ ]:
test_end_aggr.astype(int)

In [ ]:
np.allclose(
    test_preds[23] * (0.5 / 0.75)  + test_preds[24] * (0.25 / 0.75),
    test_preds_aggr[12]
)

In [ ]:
np.allclose(
    test_preds[26] * 0.25  + test_preds[27] * 0.5 + test_preds[28] * 0.25,
    test_preds_aggr[14]
)

In [ ]:
np.allclose(
    test_preds[228] * (0.25 / 0.75)  + test_preds[229] * (0.5 / 0.75),
    test_preds_aggr[119]
)

In [ ]:
np.where(test_dfidx == 9)

In [ ]:
test_end[np.where(test_dfidx == 9)[0]]

In [ ]:
np.where(test_dfidx_aggr == 9)

In [ ]:
test_end_aggr[np.where(test_dfidx_aggr == 9)[0]]

In [ ]:
def postprocessing_array(only_probs, top=6):
    print("Top", top)
    N, F = only_probs.shape
    only_probs = only_probs.reshape((N//12, 12, F))
    # mean_ = np.mean(np.sort(only_probs, axis=1)[:, 8:], axis=1, keepdims=True)
    mean_ = np.mean(np.sort(only_probs, axis=1)[:, -top:], axis=1, keepdims=True)
    only_probs *= mean_
    return only_probs.reshape((N, F))

In [ ]:
for mi in range(test_preds.shape[-1]):
    test_preds[:,:,mi] = postprocessing_array(test_preds[:,:,mi], top=1)

In [ ]:
test_preds[:,:,0].max(axis=1)

In [ ]:
from scipy.stats import rankdata

In [ ]:
np.apply_along_axis(rankdata, axis=0, arr=test_preds).mean(axis=2)

In [ ]:
test_pred_df_onnx = compose_submission_dataframe(
    probs=test_preds_onnx,
    dfidxs=test_dfidx_onnx,
    end_seconds=test_end_onnx,
    filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
    bird2id=bird2id
)

In [ ]:
test_pred_df_onnx = compose_submission_dataframe(
    probs=test_preds_aggr,
    dfidxs=test_dfidx_aggr,
    end_seconds=test_end_aggr,
    filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
    bird2id=bird2id
)